# Get 5 min volume for Feb 2018 from Unprocessed Speed

    Created by: Apoorba Bibeka
    Date: March 20, 2019

## Import required modules

In [1]:
import getpass
import boto3
import pandas as pd
from impala.dbapi import connect
from impala.util import as_pandas
from io import BytesIO as StringIO

## Connect to the database

In [2]:
username = "abibeka@securedatacommons.com"
password1 = getpass.getpass()

········


In [3]:
conn = connect(host="172.18.1.20",auth_mechanism='PLAIN',port=10000,user=username, password=password1)

## Look at the device ids from "wydot_speed_sensors_index"

In [4]:
cursor = conn.cursor()
cursor.execute('select * from wydot_speed_sensors_index')
# Store the index data tanle
SSindex = as_pandas(cursor)
# Remove "wydot_speed_sensors_index." from the col names
Rename1 ={x:x.split('wydot_speed_sensors_index.')[1] for x in SSindex}
SSindex=SSindex.rename(index=str,columns=Rename1)
print(SSindex['deviceid'].values,SSindex.columns)

(array([ 382,  383,  393,  394,  395,  396,  398,  400,  405,  407,  408,
        411,  384,  482, 1075, 1084, 1100, 1134, 1145, 1153, 1167, 1219,
       1231,  385, 1241, 1251, 1258, 1269, 1280, 1327, 1342, 1837, 1838,
       1839,  386, 2020, 2032, 2049, 2070, 2079, 2090, 2146, 2147, 2178,
       2191,  387, 2202, 2213, 2246, 2263, 2274, 2289, 2298, 2310, 2319,
       2334,  388, 2346, 2359, 2372, 2383, 2395, 2409, 2421, 2433, 2445,
       2578,  389, 2607, 2609, 2916, 3236, 3243, 3249, 3296, 3402, 3482,
       3654,  390, 3897, 3899, 3901, 3903, 3905, 3907, 3909, 3911,  391], dtype=int64), Index([u'deviceid', u'lat_decimal', u'long_decimal', u'road_code', u'sitename',
       u'devicename', u'sensortype', u'public_route', u'gis_route',
       u'direction', u'milepost', u'sensor_loc', u'nearest_rwis', u'rwis',
       u'backup_rwis', u'2015_adt', u'vsl_id', u'eb_vsl', u'wb_vsl',
       u'horiz_d', u'horiz_i', u'vert_i', u'vert_d', u'notes'],
      dtype='object'))


### Subset data for Lamarie & Cheyenne

In [5]:
SS2index=SSindex.loc[(SSindex['milepost']>=314)&(SSindex['milepost']<=360)]
SS2index = SS2index[["deviceid","milepost","direction","2015_adt"]].copy()
print(SS2index['deviceid'].values)

[ 383  393  394  395  396  384  385 1839  386 2146 2147 2178 2191  387 2202
 2213 2246 2263 2274 2289 2298 2310 2319  388  389 3654  390 3911  391]


## Get the index Data (314 <= Milepost <=360) from S3 bucket (Source: WyDoT Report)

In [6]:
#Data I got from a Report
client=boto3.client('s3')
# Get the data from our bucket directly
obj=client.get_object(Bucket='prod-sdc-tti-911061262852-us-east-1-bucket',Key="abibeka/Data/SpeedSen_MP_314_360.csv")
AB_SS=pd.read_csv(obj['Body'])
AB_SS.head()

,DEVICEID,SITENAME,MP,Sensor_Loc
0,396,Laramie East,317.68,EB
1,3911,Telephone Canyon,320.70,WB
2,395,Summit,322.05,WB
3,2146,Summit,322.60,WB
4,2147,Summit,323.85,EB


## Compare index from WyDOT Report and Database

In [7]:
# Return set with elements in either the set or ohter but not both
print(set(AB_SS['DEVICEID'].values) ^ set(SS2index['deviceid'].values))
# Return intersection 
print(set(AB_SS['DEVICEID'].values) & set(SS2index['deviceid'].values))

set([])
set([384, 385, 386, 387, 388, 389, 2310, 391, 393, 394, 395, 396, 2178, 2191, 2202, 3654, 1839, 2246, 3911, 2146, 2263, 2319, 390, 2213, 2274, 2147, 2289, 2298, 383])


## Get relevant columns and Rows from the Index Data

In [8]:
SS2index=SS2index.sort_values(by="milepost")
SS2index=SS2index.reset_index(drop=True)
SS2index.head()

,deviceid,milepost,direction,2015_adt
0,396,317.68,B,13890
1,3654,318.50,D,13890
2,3911,320.70,B,13890
3,395,322.05,B,13890
4,2146,322.60,B,13890


## Initial Approach to get Volume Data (Not good - Can Aggregate Directly on Hive)

In [9]:
#create_query='''SELECT t1.controller, t1.mountain, t2.direction, t2.milepost, t2.2015_adt FROM 
#(SELECT * FROM wydot_speed_unprocessed WHERE mountain between '2018-02-01' AND '2018-02-28') t1
#JOIN wydot_speed_sensors_index t2 
#ON (t1.controller = t2.deviceid)'''

#create_query='''SELECT t1.controller, t1.mountain, t2.direction, t2.milepost, t2.2015_adt FROM 
#(SELECT * FROM wydot_speed_unprocessed WHERE mountain between '2018-02-01' AND '2018-02-10') t1
#JOIN wydot_speed_sensors_index t2 
#ON (t1.controller = t2.deviceid)'''
#cursor.execute(create_query)
#Vol_dat=as_pandas(cursor)
#Vol_dat.to_csv("C:/Users/abibeka/Documents/Hive-SDC/Vol_dat.csv")
#print(cursor.fetchall())

## $2^{nd}$ Approach to get Count Data

### Specify Start and End Date for Data Aggregation

In [10]:
start_date='2018-02-01'
end_date='2018-03-01' 

In [11]:
create_query='''SELECT t1.controller, t1.Time5M,t1.NRec, t2.direction, t2.milepost, t2.2015_adt FROM 
    (SELECT controller, FROM_UNIXTIME(CEILING(UNIX_TIMESTAMP(mountain)/300)*300) AS Time5M, count(*) AS NRec
    FROM wydot_speed_unprocessed 
    WHERE mountain between '{}' AND '{}'
    GROUP BY controller, FROM_UNIXTIME(CEILING(UNIX_TIMESTAMP(mountain)/300)*300)) t1
JOIN (SELECT * FROM wydot_speed_sensors_index WHERE milepost BETWEEN 314 AND 360) t2 
ON (t1.controller = t2.deviceid)
ORDER BY t1.controller, t1.Time5M'''.format(start_date,end_date)
cursor.execute(create_query)
Vol_dat2=as_pandas(cursor)
Vol_dat2.head()

,t1.controller,t1.time5m,t1.nrec,t2.direction,t2.milepost,t2.2015_adt
0,384,2018-02-01 00:05:00,20,I,325.8,13890
1,384,2018-02-01 00:10:00,9,I,325.8,13890
2,384,2018-02-01 00:15:00,17,I,325.8,13890
3,384,2018-02-01 00:20:00,17,I,325.8,13890
4,384,2018-02-01 00:25:00,10,I,325.8,13890


In [12]:
# Store the data to Z drive
#Vol_dat2.to_csv("Z:/Apoorb/Vol_dat_V1.csv")

In [13]:
print(Vol_dat2['t1.controller'].unique(),SS2index['deviceid'].unique()) 

(array([ 384,  385,  386,  387,  388,  389,  390,  391,  396, 1839, 2146,
       2147, 2178, 2191, 2202, 2213, 2246, 2263, 2274, 2289, 2298, 2310,
       2319, 3654, 3911], dtype=int64), array([ 396, 3654, 3911,  395, 2146, 2147,  394,  383,  384,  385, 2178,
        393, 2191, 2202, 2213,  386,  387,  388,  389, 2246,  390, 2263,
        391, 2274, 2289, 2298, 2310, 2319, 1839], dtype=int64))


## Check for Missing Index

In [14]:
# Return set with sensor ids missing from the database:
print(set(SS2index['deviceid'].values)-set(Vol_dat2['t1.controller'].values))

set([393, 394, 395, 383])


## Write file directly to my Bucket

    Note: For Python 2.7 we imported BytesIO as StringIO
    Don't need to do this for Python 3.x
    Just use import StringIO as StringIO

In [15]:
csv_buffer=StringIO()
Vol_dat2.to_csv(csv_buffer,sep=",",index=False)
s3=boto3.resource('s3')
s3.Object('prod-sdc-tti-911061262852-us-east-1-bucket','abibeka/Data//Feb2018HiveVolDat.csv').put(Body=csv_buffer.getvalue())

{u'ETag': '"d0d8949f1870afaa6c47115ba52e11ec"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Thu, 18 Apr 2019 18:06:51 GMT',
   'etag': '"d0d8949f1870afaa6c47115ba52e11ec"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'HrCn5RZvd3Sgs5QFs3FyjXpzrXBUor3WX8TrTlE/v2oUmak8XMu/mW9QtIwWnP4JqPnxbUKG/Uc=',
   'x-amz-request-id': '94D6919E49F51D2C'},
  'HTTPStatusCode': 200,
  'HostId': 'HrCn5RZvd3Sgs5QFs3FyjXpzrXBUor3WX8TrTlE/v2oUmak8XMu/mW9QtIwWnP4JqPnxbUKG/Uc=',
  'RequestId': '94D6919E49F51D2C',
  'RetryAttempts': 0}}

# Get data divided by lane and vehicle type

start_date='2018-02-01'
end_date='2018-02-28'
create_query='''SELECT t1.controller, t1.lane, t1.vehclass, t1.Time5M,t1.NRec, t2.direction, t2.milepost, t2.2015_adt FROM 
    (SELECT controller, FROM_UNIXTIME(CEILING(UNIX_TIMESTAMP(mountain)/300)*300) AS Time5M, lane, vehclass, count(*) AS NRec
    FROM wydot_speed_unprocessed 
    WHERE mountain between '{}' AND '{}'
    GROUP BY controller, lane, vehclass,FROM_UNIXTIME(CEILING(UNIX_TIMESTAMP(mountain)/300)*300)) t1
JOIN (SELECT * FROM wydot_speed_sensors_index WHERE milepost BETWEEN 314 AND 360) t2 
ON (t1.controller = t2.deviceid)
ORDER BY t1.controller, t1.Time5M, t1.lane, t1.vehclass'''.format(start_date,end_date)
cursor.execute(create_query)
Vol_dat3=as_pandas(cursor)


Vol_dat3.head()

# Get volume and Speed data divided by lane and vehicle type

In [16]:
start_date='2018-02-01'
end_date='2018-03-01' 
create_query='''SELECT t1.controller, t1.lane, t1.vehclass, t1.Time5M, t1.speedmph_avg, t1.NRec, t2.direction, t2.milepost, t2.2015_adt FROM 
    (SELECT controller, FROM_UNIXTIME(CEILING(UNIX_TIMESTAMP(mountain)/300)*300) AS Time5M, lane, vehclass, AVG(speedmph) AS speedmph_avg, count(*) AS NRec
    FROM wydot_speed_unprocessed 
    WHERE mountain between '{}' AND '{}'
    GROUP BY controller, lane, vehclass,FROM_UNIXTIME(CEILING(UNIX_TIMESTAMP(mountain)/300)*300)) t1
JOIN (SELECT * FROM wydot_speed_sensors_index WHERE milepost BETWEEN 314 AND 360) t2 
ON (t1.controller = t2.deviceid)
ORDER BY t1.controller, t1.Time5M, t1.lane, t1.vehclass'''.format(start_date,end_date)
cursor.execute(create_query)
Vol_dat3=as_pandas(cursor)


In [17]:
Vol_dat3.head(500)

,t1.controller,t1.lane,t1.vehclass,t1.time5m,t1.speedmph_avg,t1.nrec,t2.direction,t2.milepost,t2.2015_adt
0,384,1,1,2018-02-01 00:05:00,NaN,1,I,325.8,13890
1,384,1,2,2018-02-01 00:05:00,NaN,1,I,325.8,13890
2,384,1,3,2018-02-01 00:05:00,63.550001,4,I,325.8,13890
3,384,3,3,2018-02-01 00:05:00,65.703334,3,I,325.8,13890
4,384,4,1,2018-02-01 00:05:00,65.650002,1,I,325.8,13890
5,384,4,2,2018-02-01 00:05:00,60.340000,1,I,325.8,13890
6,384,4,3,2018-02-01 00:05:00,58.105556,9,I,325.8,13890
7,384,1,3,2018-02-01 00:10:00,59.123333,6,I,325.8,13890
8,384,4,2,2018-02-01 00:10:00,63.280001,2,I,325.8,13890
9,384,4,3,2018-02-01 00:10:00,62.209999,1,I,325.8,13890


In [18]:
start_date='2018-02-01'
end_date='2018-03-01' 
q1='''SELECT controller, FROM_UNIXTIME(CEILING(UNIX_TIMESTAMP(mountain)/300)*300) AS Time5M, lane, vehclass, AVG(speedmph) AS speedmph_avg, count(*) AS NRec
    FROM wydot_speed_unprocessed 
    WHERE mountain between '{}' AND '{}' AND controller = 384
    GROUP BY controller, lane, vehclass,FROM_UNIXTIME(CEILING(UNIX_TIMESTAMP(mountain)/300)*300)'''.format(start_date,end_date)
cursor.execute(q1)
volDebug=as_pandas(cursor)

In [19]:
volDebug.head(50)

,controller,time5m,lane,vehclass,speedmph_avg,nrec
0,384,2018-02-01 18:25:00,1,0,NaN,1
1,384,2018-02-01 22:10:00,1,0,NaN,1
2,384,2018-02-02 06:35:00,1,0,NaN,2
3,384,2018-02-02 19:25:00,1,0,NaN,1
4,384,2018-02-03 11:10:00,1,0,19.549999,1
5,384,2018-02-03 20:25:00,1,0,NaN,1
6,384,2018-02-03 21:00:00,1,0,57.189999,1
7,384,2018-02-04 14:25:00,1,0,NaN,1
8,384,2018-02-07 10:40:00,1,0,41.279999,1
9,384,2018-02-09 03:50:00,1,0,NaN,1


In [20]:
import numpy as np
np.sum(np.isnan(Vol_dat3['t1.speedmph_avg']))

58730

# Save aggregate data by lane and vehicle class

In [21]:
csv_buffer=StringIO()
Vol_dat3.to_csv(csv_buffer,sep=",",index=False)
s3=boto3.resource('s3')
s3.Object('prod-sdc-tti-911061262852-us-east-1-bucket','abibeka/Data/Feb2018HiveVolDat_Ln_Cls.csv').put(Body=csv_buffer.getvalue())

{u'ETag': '"45f0d03a1c090b250c93b30f1ff29672"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Thu, 18 Apr 2019 18:14:04 GMT',
   'etag': '"45f0d03a1c090b250c93b30f1ff29672"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'K3zW6mwYPUCYlIFD8+v1MWYMd1CeFuFaHIc1IIhq9oVqSv/zpTCcCpEeIrBymrYR9co7ZAIKORY=',
   'x-amz-request-id': 'BD8F10631A35007F'},
  'HTTPStatusCode': 200,
  'HostId': 'K3zW6mwYPUCYlIFD8+v1MWYMd1CeFuFaHIc1IIhq9oVqSv/zpTCcCpEeIrBymrYR9co7ZAIKORY=',
  'RequestId': 'BD8F10631A35007F',
  'RetryAttempts': 0}}